In [364]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns

In [370]:
df = pd.read_csv("data/raw_csw.csv")

In [366]:
df.columns

Index(['Unnamed: 0', 'name', 'price', 'Brand', 'Collection', 'Series',
       'Model No', 'Features', 'Movement', 'Calibre', 'Case Size',
       'Case Thickness', 'Case Shape', 'Case Material', 'Case Back',
       'Glass Material', 'Luminosity', 'Dial Colour', 'Hands', 'Indexes',
       'Strap Material', 'Strap Colour', 'Clasp Type', 'Buckle/Clasp Material',
       'EAN', 'Gender', 'Water Resistance (M)', 'Warranty Period',
       'Country of Origin', 'Reference', 'Display', 'Power Reserve', 'Jewels',
       'Frequency', 'Precious Stone', 'Interchangeable Strap', 'Bezel',
       'Lug Width', 'Limited Edition', 'Diameter', 'Date', 'Chronograph',
       'Base', 'Frequency (bph)', 'Power Reserve (hours)', 'jewels',
       'Dial Type'],
      dtype='object')

In [367]:
df = df.drop(columns=['Unnamed: 0', 'name', 'Collection', 'Series', 'Calibre', 'Date', 'Chronograph', 'jewels', 'Lug Width', 'Diameter', 'Dial Type', 'Reference', 'Model No', 'EAN',
                'Case Back', 'Frequency', 'Base', 'Power Reserve (hours)', 'Dial Colour', 'Strap Colour', 'Buckle/Clasp Material'])

In [319]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5749 entries, 0 to 5748
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   price                  5749 non-null   int64  
 1   Brand                  5749 non-null   object 
 2   Features               4737 non-null   object 
 3   Movement               5742 non-null   object 
 4   Case Size              5749 non-null   float64
 5   Case Thickness         4230 non-null   float64
 6   Case Shape             5749 non-null   object 
 7   Case Material          5747 non-null   object 
 8   Glass Material         5741 non-null   object 
 9   Luminosity             3448 non-null   object 
 10  Hands                  3133 non-null   object 
 11  Indexes                3001 non-null   object 
 12  Strap Material         5749 non-null   object 
 13  Clasp Type             4673 non-null   object 
 14  Gender                 5749 non-null   object 
 15  Wate

In [320]:
def count_median(feature):
    counts = df[feature].value_counts()

    median_prices = df.groupby(feature)['price'].median().sort_values(ascending=False)

    result_df = pd.DataFrame({
        'Count': counts,
        'Median Price': median_prices
    }).sort_values(by='Median Price', ascending=False)

    return result_df

In [321]:
brand_counts = df['Brand'].value_counts()
popular_brands = brand_counts[brand_counts >= 25].index 
df['Brand'] = df['Brand'].apply(lambda x: x if x in popular_brands else "Others")

In [322]:
df['Water Resistance (M)'] = df['Water Resistance (M)'].replace('Splash Resistant', 30).astype(float)

In [323]:
df['Power Reserve'] = df['Power Reserve'].str.extract('(\d+)', expand=False).astype(float)

In [324]:
df['Jewels'] = df['Jewels'].fillna(0)

In [325]:
count_median('Glass Material')

,Count,Median Price
Glass Material,,
Sapphire Crystal,5400,322100.0
Hardened Acrylic Crystal,3,131000.0
Mineral & Sapphire Crystal,1,41200.0
Hardlex Crystal,50,33000.0
Hesalite Crystal,12,30200.0
Mineral Crystal,273,26400.0
Mineral Glass,2,26320.0


In [326]:
counts = df['Glass Material'].value_counts()
to_keep = counts[counts >= 25].index 
df['Glass Material'] = df['Glass Material'].apply(lambda x: x if x in to_keep else "Others")

In [327]:
df['Warranty Period'] = df['Warranty Period'].str.get(0).astype(float)

In [328]:
df['Bezel'] = df['Bezel'].fillna('No Bezel')

In [329]:
count_median('Precious Stone')

,Count,Median Price
Precious Stone,,
"On Case , Dial & Bracelet",1,18949000.0
"On Case , Crown & Clasp",4,9700000.0
"On Case, Dial",2,7560000.0
"On Bezel, Lugs & Crown",1,6900000.0
"On Case, Dial & Buckle",1,6100000.0
"On Case, Dial & Lugs",2,5750000.0
On Bezel & Bracelet,2,5274000.0
"On Bezel, Dial & Crown",6,4800000.0
On Bezel & Lugs,2,3120000.0


In [330]:
precious_stones_df = pd.DataFrame()
parts = ["Case", "Dial", "Bracelet", "Crown", "Clasp", "Bezel", "Lugs", "Buckle"]

for part in parts:
    precious_stones_df[f"precious_stone_on_{part}"] = df['Precious Stone'].fillna('').apply(lambda x: 1 if part in str(x) else 0)

In [331]:
precious_stones_df.head()

,precious_stone_on_Case,precious_stone_on_Dial,precious_stone_on_Bracelet,precious_stone_on_Crown,precious_stone_on_Clasp,precious_stone_on_Bezel,precious_stone_on_Lugs,precious_stone_on_Buckle
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0


In [333]:
string = ', '.join(list(df['Features'].dropna().unique()))
features = list(set(string.split(', ')))

In [334]:
features

['Thermometer',
 'Distance',
 'Activity Tracker',
 'Telemeter',
 'Bluetooth',
 'Retrograde Seconds',
 'Depthmeter',
 'Link With Smart Phone',
 'Seconds Disk',
 'Day-Night Indicator',
 'Tideograph',
 'Small Hacking Seconds',
 '1/100th Seconds',
 'Tachymeter',
 'Helium Escape Valve',
 '60 Seconds Tourbillon',
 'Month',
 'Day-Date',
 'Anti-Magnetic',
 'Day-Month',
 'Incoming Calls',
 'Multiple Time Zone',
 'Double Moon Phase',
 'Tourbillon',
 'Dual Time',
 'Retrograde',
 'Smart Watch',
 'Date',
 'Gyroscope',
 'Small 180 Seconds',
 'Multifunction',
 'Leap Year Indicator',
 'Heart Beat',
 'Compass',
 'Chronograph',
 '24 Hour Indicator',
 'Alarm',
 '1/4th seconds',
 'Small Seconds',
 'Power Saving',
 'Date Pointer',
 'GPS',
 'Small 90 Seconds',
 '1/5th Seconds',
 'Week',
 'Perpetual Calendar',
 'Equation Of Time',
 'Triple Time-zone',
 'Day',
 'Retrograde Date',
 '1/10th Seconds',
 'Calories',
 'Countdown Indicator',
 'UTC',
 'Jumping Hours',
 'Annual Calendar',
 'Flying Tourbillon',
 'Rotat

In [335]:
features_df = pd.DataFrame()

for feature in features:
    features_df[f"feature_{feature}"] = df['Features'].fillna('').apply(lambda x: 1 if feature in x else 0)

In [336]:
features_df = features_df.loc[:, features_df.apply(lambda col: col.sum() >= 10)]

In [337]:
features_df

,feature_Distance,feature_Telemeter,feature_Seconds Disk,feature_Day-Night Indicator,feature_1/100th Seconds,feature_Tachymeter,feature_Helium Escape Valve,feature_Month,feature_Day-Date,feature_Anti-Magnetic,...,feature_Flyback Chronograph,feature_Speedometer,feature_GMT,feature_Notifications,feature_Pulsometer,feature_Heart Rate Monitor,feature_World Timer,feature_Year,feature_Open Balance Wheel,feature_Moon Phase
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5744,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5745,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5746,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5747,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [338]:
for i in features_df.columns:
    print("Value count of " + i)
    print(features_df[i].value_counts())
    print("-----")

Value count of feature_Distance
feature_Distance
0    5737
1      12
Name: count, dtype: int64
-----
Value count of feature_Telemeter
feature_Telemeter
0    5738
1      11
Name: count, dtype: int64
-----
Value count of feature_Seconds Disk
feature_Seconds Disk
0    5728
1      21
Name: count, dtype: int64
-----
Value count of feature_Day-Night Indicator
feature_Day-Night Indicator
0    5712
1      37
Name: count, dtype: int64
-----
Value count of feature_1/100th Seconds
feature_1/100th Seconds
0    5739
1      10
Name: count, dtype: int64
-----
Value count of feature_Tachymeter
feature_Tachymeter
0    5328
1     421
Name: count, dtype: int64
-----
Value count of feature_Helium Escape Valve
feature_Helium Escape Valve
0    5627
1     122
Name: count, dtype: int64
-----
Value count of feature_Month
feature_Month
0    5699
1      50
Name: count, dtype: int64
-----
Value count of feature_Day-Date
feature_Day-Date
0    5422
1     327
Name: count, dtype: int64
-----
Value count of feature_An

In [339]:
df['Luminosity'].unique()

array(['On Hands, Hour Markers', nan, 'On Hands, Hour Markers & Bezel',
       'On Hands', 'On Hands & Dial', 'On Moon Phase',
       'On Hands & Hour Markers', 'Hands, Hour Markers & On Dial',
       'On Dial', 'On Hands, Moon Phase', 'Full Luminova Dial',
       'On Hands, Hour makers, On Dial, & Bezel',
       'On Hands, Hour Markers & Case'], dtype=object)

In [340]:
luminosity_df = pd.DataFrame()
parts = ['Hands', 'Hour', 'Bezel', 'Dial']

for part in parts:
    luminosity_df[f"luminosity_on_{part}"] = df['Luminosity'].fillna('').apply(lambda x: 1 if part in x else 0)

In [341]:
for i in luminosity_df.columns:
    print("Value count of " + i)
    print(luminosity_df[i].value_counts())
    print("-----")

Value count of luminosity_on_Hands
luminosity_on_Hands
1    3421
0    2328
Name: count, dtype: int64
-----
Value count of luminosity_on_Hour
luminosity_on_Hour
1    2919
0    2830
Name: count, dtype: int64
-----
Value count of luminosity_on_Bezel
luminosity_on_Bezel
0    4944
1     805
Name: count, dtype: int64
-----
Value count of luminosity_on_Dial
luminosity_on_Dial
0    5686
1      63
Name: count, dtype: int64
-----


In [342]:
df = df.drop(columns=['Luminosity', 'Features', 'Precious Stone'])

In [343]:
count_median("Case Shape")
# Nothing to change here

,Count,Median Price
Case Shape,,
Barrel,25,5670000.0
Octagon,83,1314000.0
Drop,21,955000.0
Hexagon,4,510000.0
Cushion,281,419000.0
Square,177,400000.0
Dodecagon,87,334000.0
Round,4775,280000.0
Tonneau,117,183500.0


In [344]:
coatings = ['DLC', 'PVD', 'CVD']

def extract_coating(material):
    for coating in coatings:
        if coating in material:
            return coating
    return None

df['Case Material Coating'] = df['Case Material'].fillna('').apply(extract_coating)

In [345]:
def impute_gold_enhanced_steel(material):
    if 'Gold' in material and 'Steel' in material:
        return 'G-S Hybrid'
    return material 

base_materials = ['Titanium', 'Platinum', 'Gold', 'Ceramic', 'Steel', 'Bronze', 'Aluminium']

def extract_base(material):
    if material is None:
        return None 
    for base in base_materials:
        if base in material:
            return base
    return None

def extract_carbon_based(material):
    if material is None:
        return None 
    if 'Carbo' in material or 'NORTEQ®' in material:
        return 'Carbon-based'
    return None

to_keep = ['Aluminium', 'Gold', 'Steel', 'Platinum', 'Titanium', 'Bronze', 'G-S Hybrid', 'Ceramic', 'Carbon-based']

In [346]:
def process_case_material(df):
    df['Case Material'] = df['Case Material'].fillna('').apply(impute_gold_enhanced_steel)
    df['Case Material'] = df['Case Material'].apply(extract_base).fillna(df['Case Material'])
    df['Case Material'] = df['Case Material'].apply(extract_carbon_based).fillna(df['Case Material'])
    df['Case Material'] = df['Case Material'].apply(lambda x: x if x in to_keep else "Others") 
    return df

In [347]:
df = process_case_material(df)

In [348]:
materials = ['Gold', 'Rubber', 'Leather', 'Titanium', 'Steel', 'Ceramic']

def impute_(material):
    for mate in materials:
        if mate in material:
            return mate
    return None

strap_mapping = {
    'Caoutchouc' : 'Rubber',
    'Sailcloth' : 'Fabric',
    'Nordura' : 'Fabric',
    'Silicon' : 'Silicone',
    'Calfskin' : 'Leather',
    'Cordura®' : 'Fabric',
    'Nato' : 'Nylon',
    'Cotton' : 'Fabric',
    'Textile' : 'Fabric',
    'Velvet' : 'Fabric',
    'Alcantara' : 'Fabric',
}

to_keep = ['Satin', 'Gold', 'Steel', 'Farbic', 'Leather', 'Rubber', 'Titanium', 'Nylon', 'Bronze', 'G-S Hybrid', 'Silicone', 'Ceramic']

In [349]:
def process_strap_materials(df):
    df['Strap Material'] = df['Strap Material'].apply(impute_gold_enhanced_steel)
    df['Strap Material'] = df['Strap Material'].apply(impute_).fillna(df['Strap Material'])
    df['Strap Material'] = df['Strap Material'].map(strap_mapping).fillna(df['Strap Material'])
    df['Strap Material'] = df['Strap Material'].apply(lambda x: x if x in to_keep else "Others") 
    return df

In [350]:
df = process_strap_materials(df)

In [351]:
def process_clasp(type_):
    if 'Deploy' in type_ or 'Foldover' in type_:
        return 'Deployemnt'
    elif 'Folding' in type_:
        return 'Folding'
    elif 'Butterfly' in type_:
        return 'Butterfly'
    elif 'Hook-and-loop' in type_:
        return 'Hook-and-loop'
    elif 'Tang' in type_ or 'Pin' in type_ or 'Ardillon' in type_:
        return 'Tang'
    
to_keep = ['Deployment', 'Folding', 'Butterfly', 'Hook-and-loop', 'Tang', 'Tudor “T-fit” Clasp', 'Winged Clasp']

In [352]:
df['Clasp Type'] = df['Clasp Type'].fillna('').apply(process_clasp).fillna(df['Clasp Type'])
df['Clasp Type'] = df['Clasp Type'].apply(lambda x: x if x in to_keep else "Others") 

In [353]:
for column in df.select_dtypes(include=['object']).columns:
    print('Unique values of ' + column)
    print(df[column].unique())
    print('-----------')
    print(' ')

Unique values of Brand
['Girard-Perregaux' 'Zenith' 'Omega' 'BVLGARI' 'Oris' 'Rado'
 'Raymond Weil' 'Frederique Constant' 'Breitling' 'Longines'
 'H. Moser & Cie.' 'Arnold & Son' 'Carl F. Bucherer' 'Nomos Glashutte'
 'Baume & Mercier' 'Doxa' 'Tissot' 'Parmigiani' 'Maurice Lacroix' 'Seiko'
 'Bremont' 'TAG Heuer' 'Favre Leuba' 'Corum' 'Grand Seiko'
 'Jaeger-LeCoultre' 'Hublot' 'Louis Erard' 'IWC' 'Others' 'Bell & Ross'
 'Maserati' 'Titoni' 'Panerai' 'MeisterSinger' 'Ulysse Nardin' 'Alpina'
 'Junghans' 'Zeppelin' 'Louis Moinet' 'Luminox' 'Bovet' 'NORQAIN'
 'Jacob & Co.' 'Tutima Glashütte' 'Edox' 'Perrelet' 'Tudor'
 'Nivada Grenchen' 'Gerald Charles' 'Eberhard & Co.' 'Mühle-Glashütte'
 'Ernest Borel' 'CVSTOS' 'CIGA Design' 'Edouard Koehn']
-----------
 
Unique values of Movement
['Automatic' 'Quartz' 'Manual Winding' 'Spring Drive' 'Kinetic Powered'
 'Solar Powered' 'SuperQuartz™' nan]
-----------
 
Unique values of Case Shape
['Octagon' 'Round' 'Square' 'Cushion' 'Rectangular' 'Oval' 'Ton

In [361]:
df = pd.concat([df, features_df, luminosity_df,precious_stones_df], axis=1)

In [363]:
df.to_csv("data/cleaned_v1_data.csv", index=False)